## Demonstration of the User-Based Collaborative Recommender

This system leverages collaborative filtering by analyzing user interactions, such as scroll length and read time, to identify users with similar behavior. 
Therefore, it focuses on the user-item relation.

It recommends articles that these similar users have engaged with, aiming to provide personalized suggestions. The model's performance is evaluated using MAP@K and NDCG@K metrics.



In [1]:
import sys
import os

parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(parent_dir)

import polars as pl
import numpy as np

from parquet_data_reader import ParquetDataReader
from models.user_based import CollaborativeRecommender

pl.Config.set_tbl_cols(-1)

polars.config.Config

## Data Import and EDA

In [2]:
data_reader = ParquetDataReader()
articles_df = data_reader.read_data('../../data/articles.parquet')
train_behaviors_df = data_reader.read_data('../../data/train/behaviors.parquet')
train_history_df = data_reader.read_data('../../data/train/history.parquet')
document_vectors_df = data_reader.read_data('../../data/document_vector.parquet')

We check the size of the data. From the sizes we learn that:
<ol>
  <li>We have 20738 unique articles</li>
  <li>We have 15143 unique users</li>
  <li>We have 232887 interactions in the testset</li>
</ol> 

In [3]:
print("articles_df has the size:         ", articles_df.shape)
print("train_behaviors_df has the size:  ", train_behaviors_df.shape)
print("train_history_df has the size:    ", train_history_df.shape)
print("document_vectors_df has the size: ", document_vectors_df.shape)

articles_df has the size:          (20738, 21)
train_behaviors_df has the size:   (232887, 17)
train_history_df has the size:     (15143, 5)
document_vectors_df has the size:  (125541, 2)


### Validation Set

In [4]:
test_behaviours_df = data_reader.read_data('../../data/validation/behaviors.parquet')
test_behaviours_df.head()

impression_id,article_id,impression_time,read_time,scroll_percentage,device_type,article_ids_inview,article_ids_clicked,user_id,is_sso_user,gender,postcode,age,is_subscriber,session_id,next_read_time,next_scroll_percentage
u32,i32,datetime[μs],f32,f32,i8,list[i32],list[i32],u32,bool,i8,i8,i8,bool,u32,f32,f32
96791,null,2023-05-28 04:21:24,9.0,null,2,"[9783865, 9784591, … 9784710]",[9784696],22548,false,null,null,null,false,142,72.0,100.0
96798,null,2023-05-28 04:31:48,46.0,null,2,"[9782884, 9783865, … 9784648]",[9784281],22548,false,null,null,null,false,143,16.0,28.0
96801,null,2023-05-28 04:30:17,14.0,null,2,"[9784648, 7184889, … 9781983]",[9784444],22548,false,null,null,null,false,143,12.0,24.0
96808,null,2023-05-28 04:27:19,22.0,null,2,"[9784607, 9695098, … 9781983]",[9781983],22548,false,null,null,null,false,142,125.0,80.0
96810,null,2023-05-28 04:29:47,23.0,null,2,"[9781983, 7184889, … 9781520]",[9784642],22548,false,null,null,null,false,142,null,null


In [5]:
# Combine train and test behaviors
combined_df = pl.concat([train_behaviors_df, test_behaviours_df])

# Generate a random mask for splitting
n = combined_df.height  # Total number of rows
test_mask = np.random.rand(n) < 0.30  # 30% test, 70% train

# Apply the mask
test_behaviors_df = combined_df.filter(test_mask)
train_behaviors_df = combined_df.filter(~test_mask)

# Verify the split
print(f"Train size: {train_behaviors_df.shape[0]}, Test size: {test_behaviors_df.shape[0]}")

Train size: 333987, Test size: 143547


### Table Contents

The information on news articles. As we are going to perform user-user CF, this table is not neccesary

In [6]:
articles_df.head()

article_id,title,subtitle,last_modified_time,premium,body,published_time,image_ids,article_type,url,ner_clusters,entity_groups,topics,category,subcategory,category_str,total_inviews,total_pageviews,total_read_time,sentiment_score,sentiment_label
i32,str,str,datetime[μs],bool,str,datetime[μs],list[i64],str,str,list[str],list[str],list[str],i16,list[i16],str,i32,i32,f32,f32,str
3001353,"""Natascha var ikke den første""","""Politiet frygter nu, at Natasc…",2023-06-29 06:20:33,false,"""Sagen om den østriske Natascha…",2006-08-31 08:06:45,[3150850],"""article_default""","""https://ekstrabladet.dk/krimi/…",[],[],"[""Kriminalitet"", ""Personfarlig kriminalitet""]",140,[],"""krimi""",null,null,null,0.9955,"""Negative"""
3003065,"""Kun Star Wars tjente mere""","""Biografgængerne strømmer ind f…",2023-06-29 06:20:35,false,"""Vatikanet har opfordret til at…",2006-05-21 16:57:00,[3006712],"""article_default""","""https://ekstrabladet.dk/underh…",[],[],"[""Underholdning"", ""Film og tv"", ""Økonomi""]",414,"[433, 434]","""underholdning""",null,null,null,0.846,"""Positive"""
3012771,"""Morten Bruun fyret i Sønderjys…","""FODBOLD: Morten Bruun fyret me…",2023-06-29 06:20:39,false,"""Kemien mellem spillerne i Supe…",2006-05-01 14:28:40,[3177953],"""article_default""","""https://ekstrabladet.dk/sport/…",[],[],"[""Erhverv"", ""Kendt"", … ""Ansættelsesforhold""]",142,"[196, 199]","""sport""",null,null,null,0.8241,"""Negative"""
3023463,"""Luderne flytter på landet""","""I landets tyndest befolkede om…",2023-06-29 06:20:43,false,"""Det frække erhverv rykker på l…",2007-03-24 08:27:59,[3184029],"""article_default""","""https://ekstrabladet.dk/nyhede…",[],[],"[""Livsstil"", ""Erotik""]",118,[133],"""nyheder""",null,null,null,0.7053,"""Neutral"""
3032577,"""Cybersex: Hvornår er man utro?""","""En flirtende sms til den flott…",2023-06-29 06:20:46,false,"""De fleste af os mener, at et t…",2007-01-18 10:30:37,[3030463],"""article_default""","""https://ekstrabladet.dk/sex_og…",[],[],"[""Livsstil"", ""Partnerskab""]",565,[],"""sex_og_samliv""",null,null,null,0.9307,"""Neutral"""


Each file consists of seven days of impression logs. The train_behaviors_df table contains all information about interactions between users and items, and can be used as a basis for user-user CF. <strong>Therefore we only need this table</strong>.

In [7]:
train_behaviors_df.head()

impression_id,article_id,impression_time,read_time,scroll_percentage,device_type,article_ids_inview,article_ids_clicked,user_id,is_sso_user,gender,postcode,age,is_subscriber,session_id,next_read_time,next_scroll_percentage
u32,i32,datetime[μs],f32,f32,i8,list[i32],list[i32],u32,bool,i8,i8,i8,bool,u32,f32,f32
149474,null,2023-05-24 07:47:53,13.0,null,2,"[9778623, 9778682, … 9778728]",[9778657],139836,false,null,null,null,false,759,7.0,22.0
153071,9778623,2023-05-24 07:11:08,125.0,100.0,1,"[9777492, 9774568, … 9775990]",[9777492],151570,false,null,null,null,false,1976,26.0,100.0
153075,9777492,2023-05-24 07:13:58,26.0,100.0,1,"[9778500, 9776420, … 9020783]",[9777034],151570,false,null,null,null,false,1976,7.0,16.0
153078,9777492,2023-05-24 07:13:46,7.0,100.0,1,"[9778021, 9778627, … 7213923]",[9778226],151570,false,null,null,null,false,1976,4.0,21.0
155587,9778627,2023-05-24 07:56:30,50.0,100.0,1,"[9777397, 9759955, … 9778369]",[9778375],161621,false,null,null,null,false,3625,119.0,100.0


Each file consists of users' click histories collected over 21 days period. This table does contain the same values as the train_behaviours_df, but as that table is easier to work with we will use train_behaviours_df over this one

In [8]:
train_history_df.head()

user_id,impression_time_fixed,scroll_percentage_fixed,article_id_fixed,read_time_fixed
u32,list[datetime[μs]],list[f32],list[i32],list[f32]
13538,"[2023-04-27 10:17:43, 2023-04-27 10:18:01, … 2023-05-17 20:36:34]","[100.0, 35.0, … 100.0]","[9738663, 9738569, … 9769366]","[17.0, 12.0, … 16.0]"
14241,"[2023-04-27 09:40:18, 2023-04-27 09:40:33, … 2023-05-17 17:08:41]","[100.0, 46.0, … 100.0]","[9738557, 9738528, … 9767852]","[8.0, 9.0, … 12.0]"
20396,"[2023-04-27 12:30:44, 2023-04-27 12:31:34, … 2023-05-17 10:59:44]","[100.0, 59.0, … 13.0]","[9738760, 9738355, … 9769679]","[49.0, 34.0, … 4.0]"
34912,"[2023-04-29 07:12:49, 2023-04-29 13:01:18, … 2023-05-18 05:06:40]","[100.0, 35.0, … 27.0]","[9741802, 9741804, … 9770882]","[153.0, 7.0, … 5.0]"
37953,"[2023-04-27 19:17:10, 2023-04-27 19:17:27, … 2023-05-17 21:29:22]","[14.0, 28.0, … 18.0]","[9739205, 9739202, … 9769306]","[4.0, 16.0, … 6.0]"


List of vectors for each article. This is used to describe the items. It could be used for item-item CF, but is not relevant to user-user CF.  <strong>This table will therefore not be used</strong>

In [9]:
document_vectors_df.head()

article_id,document_vector
i32,list[f32]
3000022,"[0.065424, -0.047425, … 0.035706]"
3000063,"[0.028815, -0.000166, … 0.027167]"
3000613,"[0.037971, 0.033923, … 0.063961]"
3000700,"[0.046524, 0.002913, … 0.023423]"
3000840,"[0.014737, 0.024068, … 0.045991]"


From the analasys we see that we only need train_behaviour_df to perform user-user CF

## Preprocessing

### Remove Non-Needed Values

We see that we have several items that are not required for performing user-user CF

In [10]:
train_behaviors_df.head()

impression_id,article_id,impression_time,read_time,scroll_percentage,device_type,article_ids_inview,article_ids_clicked,user_id,is_sso_user,gender,postcode,age,is_subscriber,session_id,next_read_time,next_scroll_percentage
u32,i32,datetime[μs],f32,f32,i8,list[i32],list[i32],u32,bool,i8,i8,i8,bool,u32,f32,f32
149474,null,2023-05-24 07:47:53,13.0,null,2,"[9778623, 9778682, … 9778728]",[9778657],139836,false,null,null,null,false,759,7.0,22.0
153071,9778623,2023-05-24 07:11:08,125.0,100.0,1,"[9777492, 9774568, … 9775990]",[9777492],151570,false,null,null,null,false,1976,26.0,100.0
153075,9777492,2023-05-24 07:13:58,26.0,100.0,1,"[9778500, 9776420, … 9020783]",[9777034],151570,false,null,null,null,false,1976,7.0,16.0
153078,9777492,2023-05-24 07:13:46,7.0,100.0,1,"[9778021, 9778627, … 7213923]",[9778226],151570,false,null,null,null,false,1976,4.0,21.0
155587,9778627,2023-05-24 07:56:30,50.0,100.0,1,"[9777397, 9759955, … 9778369]",[9778375],161621,false,null,null,null,false,3625,119.0,100.0


All information that does not describe a user, or a user-item interaction can therefore be removed

In [11]:
irelevant_columns = ["impression_time", "device_type", "article_ids_inview", "article_ids_clicked", "session_id", "next_read_time", "next_scroll_percentage"]
train_behaviors_df = train_behaviors_df.drop(irelevant_columns)
train_behaviors_df.head()

impression_id,article_id,read_time,scroll_percentage,user_id,is_sso_user,gender,postcode,age,is_subscriber
u32,i32,f32,f32,u32,bool,i8,i8,i8,bool
149474,null,13.0,null,139836,false,null,null,null,false
153071,9778623,125.0,100.0,151570,false,null,null,null,false
153075,9777492,26.0,100.0,151570,false,null,null,null,false
153078,9777492,7.0,100.0,151570,false,null,null,null,false
155587,9778627,50.0,100.0,161621,false,null,null,null,false


The remaining items are the ones that can be used. But already here we see that we have several features with lacking information. We should therefore treat this

### Account for Missing Values

We see here that alot of the behaviours contain missing values. The therefore have to either remove or replace the values

In [12]:
print(train_behaviors_df.shape)
train_behaviors_df.null_count()

(333987, 10)


impression_id,article_id,read_time,scroll_percentage,user_id,is_sso_user,gender,postcode,age,is_subscriber
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,234901,0,236457,0,0,311213,327610,325261,0


In [13]:
train_behaviors_df = train_behaviors_df.filter(train_behaviors_df["article_id"].is_not_null())
print(train_behaviors_df.shape)
train_behaviors_df.null_count()

(99086, 10)


impression_id,article_id,read_time,scroll_percentage,user_id,is_sso_user,gender,postcode,age,is_subscriber
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,2599,0,0,92197,96831,95673,0


We see that of 70421, are there between 65-68000 missing values for gender, postcode and age. We therefore remove these as there is no use subsidizing them

In [14]:
train_behaviors_df = train_behaviors_df.drop(["gender", "postcode", "age"])
print(train_behaviors_df.shape)
train_behaviors_df.null_count()

(99086, 7)


impression_id,article_id,read_time,scroll_percentage,user_id,is_sso_user,is_subscriber
u32,u32,u32,u32,u32,u32,u32
0,0,0,2599,0,0,0


We still see that 2570/98967 rows are missing a scroll percentage. As this is very low (<3%) we can easily replace this. Intitially we just set scroll to 0

In [15]:
train_behaviors_df = train_behaviors_df.fill_null(strategy="zero")

### Account for Multiple Instances of the Same Article and User

By checking rows where the user_id and article_id are the same we see that we have 9855 instances where the user has read the same article multiple times

In [16]:
duplicates = train_behaviors_df.group_by(["article_id", "user_id"]).count().filter(pl.col("count") > 1)

print(duplicates)

shape: (12_281, 3)
┌────────────┬─────────┬───────┐
│ article_id ┆ user_id ┆ count │
│ ---        ┆ ---     ┆ ---   │
│ i32        ┆ u32     ┆ u32   │
╞════════════╪═════════╪═══════╡
│ 9779517    ┆ 2279550 ┆ 2     │
│ 9788862    ┆ 651964  ┆ 5     │
│ 9789616    ┆ 1009047 ┆ 2     │
│ 9785732    ┆ 2120961 ┆ 2     │
│ 9789911    ┆ 1998759 ┆ 2     │
│ …          ┆ …       ┆ …     │
│ 9790942    ┆ 2404006 ┆ 2     │
│ 9783278    ┆ 1631821 ┆ 3     │
│ 9774542    ┆ 1447939 ┆ 2     │
│ 9780860    ┆ 682266  ┆ 2     │
│ 9776916    ┆ 72717   ┆ 2     │
└────────────┴─────────┴───────┘


C:\Users\chris\AppData\Local\Temp\ipykernel_984\4233841252.py:1: DeprecationWarning: `GroupBy.count` is deprecated. It has been renamed to `len`.
  duplicates = train_behaviors_df.group_by(["article_id", "user_id"]).count().filter(pl.col("count") > 1)


We see that we need to combine these duplicate rows. We therefore propose that for multiple instances of the same article and user, we combine the readtime and select the largest scroll percentage. This way we can preserve the data without having duplicates

In [17]:
train_behaviors_df = (
    train_behaviors_df
    .group_by(["article_id", "user_id"])
    .agg(
        pl.col("read_time").product().alias("total_readtime"),  # Multiply all readtime values
        pl.col("scroll_percentage").max().alias("max_scroll")  # Select the largest scroll percentage
    )
)

print(train_behaviors_df)

shape: (79_466, 4)
┌────────────┬─────────┬────────────────┬────────────┐
│ article_id ┆ user_id ┆ total_readtime ┆ max_scroll │
│ ---        ┆ ---     ┆ ---            ┆ ---        │
│ i32        ┆ u32     ┆ f32            ┆ f32        │
╞════════════╪═════════╪════════════════╪════════════╡
│ 9789600    ┆ 184630  ┆ 20.0           ┆ 100.0      │
│ 9779777    ┆ 61776   ┆ 58.0           ┆ 100.0      │
│ 9781906    ┆ 707243  ┆ 1722.0         ┆ 100.0      │
│ 9777767    ┆ 2121561 ┆ 173.0          ┆ 0.0        │
│ 9775964    ┆ 2264003 ┆ 32.0           ┆ 100.0      │
│ …          ┆ …       ┆ …              ┆ …          │
│ 9783752    ┆ 2124286 ┆ 10.0           ┆ 100.0      │
│ 9773278    ┆ 1890632 ┆ 187.0          ┆ 100.0      │
│ 9787465    ┆ 1049790 ┆ 21.0           ┆ 100.0      │
│ 9781796    ┆ 82327   ┆ 38.0           ┆ 100.0      │
│ 9785062    ┆ 1998503 ┆ 48.0           ┆ 100.0      │
└────────────┴─────────┴────────────────┴────────────┘


## Model Fit

This first model uses readtime and read percentage interactions to compare the user interactions 

In [18]:
recommender = CollaborativeRecommender(train_behaviors_df)
recommender.fit()

{184630: [(2384701, np.float64(0.910121060246197)),
  (2307527, np.float64(0.8994855238548193)),
  (1667838, np.float64(0.8994855238548192)),
  (2202747, np.float64(0.8994855238548192)),
  (2239705, np.float64(0.8994855238548192)),
  (139029, np.float64(0.8994855238548192)),
  (1103841, np.float64(0.8994844339404159)),
  (1499245, np.float64(0.8994634092896541)),
  (1823980, np.float64(0.8993848923267967)),
  (653653, np.float64(0.8988783681754189))],
 61776: [(2500143, np.float64(0.25871745759640086)),
  (721152, np.float64(0.2546056412170262)),
  (191640, np.float64(0.25057002314040555)),
  (2140995, np.float64(0.2504957872377729)),
  (1585138, np.float64(0.2504957872377729)),
  (1899167, np.float64(0.2504956232575948)),
  (2322519, np.float64(0.2493932873403617)),
  (940349, np.float64(0.24081998429725804)),
  (1274794, np.float64(0.24081998429725804)),
  (1553664, np.float64(0.24081998429725804))],
 707243: [(576685, np.float64(0.9767787279298252)),
  (1455620, np.float64(0.9767787

This first model just compares all artilces read by users when comparing users

In [19]:
binary_recommender = CollaborativeRecommender(train_behaviors_df, binary_model=True)
binary_recommender.fit()

{184630: [(1707197, np.float64(0.35355339059327373)),
  (625667, np.float64(0.35355339059327373)),
  (1151547, np.float64(0.28867513459481287)),
  (2384701, np.float64(0.28867513459481287)),
  (103826, np.float64(0.28347335475692037)),
  (455549, np.float64(0.28347335475692037)),
  (2369007, np.float64(0.25)),
  (2202747, np.float64(0.25)),
  (1196547, np.float64(0.25)),
  (2073394, np.float64(0.25))],
 61776: [(437153, np.float64(0.22086305214969304)),
  (1821915, np.float64(0.22086305214969304)),
  (991399, np.float64(0.22086305214969304)),
  (2500143, np.float64(0.22086305214969304)),
  (1550400, np.float64(0.22086305214969304)),
  (402858, np.float64(0.22086305214969304)),
  (1543165, np.float64(0.22086305214969304)),
  (1250794, np.float64(0.22086305214969304)),
  (639734, np.float64(0.22086305214969304)),
  (2274313, np.float64(0.22086305214969304))],
 707243: [(1389246, np.float64(0.5)),
  (943206, np.float64(0.5)),
  (272127, np.float64(0.5)),
  (1455620, np.float64(0.5)),
  (5

Of the original 15143 users, only 9194 can be accounted for with the current solution. This should be changed in the future

## Model Presentation

### Article Recommendation

In [20]:
for user in [630220, 620796, 1067393, 1726258, 17205]:
    print("reccomended for user ", user, ": ", recommender.recommend_n_articles(user_id=user, n=5, allow_read_articles=True))

reccomended for user  630220 :  [9778351, 9772453, 9771554, 9781389, 9776147]
reccomended for user  620796 :  [9783865, 9788197, 9787176, 9783379, 9771995]
reccomended for user  1067393 :  [9787098, 9776508, 9786906, 9773341, 9789494]
reccomended for user  1726258 :  []
reccomended for user  17205 :  [9771125, 9773744, 9779748, 9783657, 9789473]


In [21]:
for user in [630220, 620796, 1067393, 1726258, 17205]:
    print("reccomended for user ", user, ": ", binary_recommender.recommend_n_articles(user_id=user, n=5, allow_read_articles=True))

reccomended for user  630220 :  [9783334, 9775484, 9772355, 9789473, 9788524]
reccomended for user  620796 :  [9783278, 9771168, 9783334, 9779737, 9783379]
reccomended for user  1067393 :  [9787098, 9790811, 9782746, 9744403, 9775277]
reccomended for user  1726258 :  []
reccomended for user  17205 :  [9775562, 9780325, 9771846, 9775277, 9771333]


### Evaluation Scores

#### Without the Ability to Recommend Read Articles

The complex model only reccomending articles the user has not yet read

In [22]:
results = recommender.evaluate_recommender(test_behaviours_df, k=100, n_jobs=4, user_sample=200, allow_read_articles=False)
results

{'MAP@K': np.float64(0.0008196721311475411),
 'NDCG@K': np.float64(0.0036933479774460052)}

The binary reccomender model only reccomending articles the user has not yet read

In [23]:
results = binary_recommender.evaluate_recommender(test_behaviours_df, k=100, n_jobs=4, user_sample=200, allow_read_articles=False)
results

{'MAP@K': np.float64(0.0004022988505747126),
 'NDCG@K': np.float64(0.0007428582529643576)}

#### With the Ability to Recommend Previously Read Articles

The complex model reccomending articles the user, even if they have read them before

In [24]:
results = recommender.evaluate_recommender(test_behaviours_df, k=100, n_jobs=4, user_sample=200, allow_read_articles=True)
results

{'MAP@K': np.float64(0.010877192982456142),
 'NDCG@K': np.float64(0.20570205521383123)}

The binary reccomender model reccomending articles the user, even if they have read them before

In [25]:
results = binary_recommender.evaluate_recommender(test_behaviours_df, k=100, n_jobs=4, user_sample=200, allow_read_articles=True)
results

{'MAP@K': np.float64(0.02188235294117647),
 'NDCG@K': np.float64(0.33195101297246943)}

## Model Experimentation

In [26]:
test_user_id = 630220

predictions = recommender.recommend_n_articles(user_id=test_user_id, n=1000, allow_read_articles=True)
results = set(test_behaviours_df.filter(pl.col("user_id") == test_user_id)["article_id"])

print(results)
print(predictions)

for prediction in predictions:
    if prediction in results:
        print("Yes")

{9786243, 9787524, 9781902, 9784591, 9783824, 9786111, 9776916, 9779615, 9788705, 9789473, 9428643, 9783334, 9782315, 9756075, 9787441, 9782722, 9786821, 9782726, 9786566, 9789896, 9787465, 9788362, 9791049, None, 9782092, 9780815, 9783509, 9772508, 9786718, 9786719, 9787487, 9790942, 9783655, 9786351, 9780849, 9781875, 9788661, 9781878, 9787510, 9786618, 9673979, 9780348, 9781887}
[9778351, 9772453, 9771554, 9781389, 9776147, 9778945, 9769557, 9772300, 9782869, 9779289, 9771168, 9783657, 9781998, 9777811, 9783993, 9774430, 9765759, 9773846, 9780280, 9774120, 9786378, 9776544, 9778139, 9779269, 9771330, 9785471, 9778413, 9779408, 9772343, 9771253, 9780677, 9776715, 9779650, 9774527, 9779737, 9784863, 9779724, 9769624, 9778915, 9780406, 9771919, 9775881, 9772227, 9780960, 9772475, 9789997, 9779045, 9771350, 9783057, 9784879, 9775484, 9777319, 9780561, 9773210, 9773543, 9784710, 9779294, 9771187, 9772442, 9774187, 9776855, 9780496, 9771170, 9760091, 9781423, 9785475]


In [27]:
test_user_id = 630220

predictions = recommender.recommend_n_articles(user_id=test_user_id, n=1000, allow_read_articles=True)
results = set(test_behaviours_df.filter(pl.col("user_id") == test_user_id)["article_id"])

print(results)
print(predictions)

for prediction in predictions:
    if prediction in results:
        print("Yes")

{9786243, 9787524, 9781902, 9784591, 9783824, 9786111, 9776916, 9779615, 9788705, 9789473, 9428643, 9783334, 9782315, 9756075, 9787441, 9782722, 9786821, 9782726, 9786566, 9789896, 9787465, 9788362, 9791049, None, 9782092, 9780815, 9783509, 9772508, 9786718, 9786719, 9787487, 9790942, 9783655, 9786351, 9780849, 9781875, 9788661, 9781878, 9787510, 9786618, 9673979, 9780348, 9781887}
[9778351, 9772453, 9771554, 9781389, 9776147, 9778945, 9769557, 9772300, 9782869, 9779289, 9771168, 9783657, 9781998, 9777811, 9783993, 9774430, 9765759, 9773846, 9780280, 9774120, 9786378, 9776544, 9779269, 9778139, 9771330, 9785471, 9778413, 9779408, 9772343, 9771253, 9780677, 9776715, 9779650, 9774527, 9779737, 9784863, 9778915, 9779724, 9769624, 9780406, 9771919, 9775881, 9772227, 9780960, 9772475, 9789997, 9779045, 9771350, 9783057, 9784879, 9775484, 9777319, 9773210, 9780561, 9773543, 9784710, 9779294, 9771187, 9772442, 9774187, 9776855, 9780496, 9771170, 9760091, 9781423, 9785475]
